In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools as it
import transformers
import torch
import torch.nn.functional as F


/home/sviatoslav/anaconda3/envs/vllm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cipher = """Xoi Doijtooi Tooijto-Ntoit-oo Toooj bot iott boi Giüobooj boi Öotoiooboooi to Jobi 4014 toit 40.000 Nojoioojojoo oot bto Xäoboi noo Ditnottooboo to Xootiobjoob joiobioopt. Dti zoo Jobioioobo iojjoo oi ioob 100.000 Dojojoo ioto. 4044 taootoo ooob otoooj io ntojo Nojoibäoboi btozotooooo. Xoiüpoi btoooi bot Toooj poiotti Iooioobo Tooijtoiootoboi, Loboitottoooo tüi Tjottioootoi oob Uäioooooooo pot iotooo Kooboo toitojjtoit. Noboo to oäobitoo Jobi taooto oto Iotj btoioi Dojojoo oji otooi boi jiaßtoo ntitoojjoo Kiottnoito to Toiooo oo Tooijtoooitt oottiotoo.
"""
text = """Das Berliner Energie-Start-up Enpal hat seit der Gründung des Unternehmens im Jahr 2017 fast 70.000 Solaranlagen auf die Dächer von Privatkunden in Deutschland geschraubt. Bis zum Jahresende sollen es rund 100.000 Anlagen sein. 2025 könnten noch einmal so viele Solardächer hinzukommen. Darüber hinaus hat Enpal bereits Tausende Energiespeicher, Ladestationen für Elektroautos und Wärmepumpen bei seinen Kunden installiert. Schon im nächsten Jahr könnte ein Teil dieser Anlagen als eines der größten virtuellen Kraftwerke in Europa am Energiemarkt auftreten.
"""

In [3]:
cipher_list = cipher.lower().split(' ')
cipher_list = [word.strip('.').strip('\n') for word in cipher_list ] 

text_list = text.lower().split(' ')
text_list = [word.strip('.').strip('\n') for word in text_list ] 

pairs = {(i, k) for i,k in zip(cipher_list, text_list)}
rules = {}

for pair in pairs:
    for c, s in zip(pair[0],pair[1]):
        if c in rules:
            if s not in rules[c]:
                rules[c] += s
        else:
            rules[c] = s
rules

{'o': 'enmaupoc',
 't': 'ietfk',
 'j': 'lgj',
 'd': 'bpa',
 'i': 'rst',
 '-': '-',
 'n': 'svw',
 '4': '275',
 '0': '0',
 '1': '1',
 'b': 'hd',
 'ü': 'ü',
 'z': 'z',
 'p': 'b',
 '.': '.',
 'ä': 'ä',
 'a': 'ö',
 'ß': 'ß',
 'x': 'd',
 'u': 'w',
 ',': ',',
 'g': 'g',
 'l': 'l',
 'k': 'k',
 'ö': 'u'}

In [4]:
abc = ''.join([chr(ord('a')+i) for i in range(ord('z') - ord('a')+1)])
for i in range(ord('z') - ord('a')):
    ch = chr(ord('a')+i)
    if ch not in rules:
        print(ch)
        rules[ch] = abc

c
e
f
h
m
q
r
s
v
w
y


In [5]:
rules

{'o': 'enmaupoc',
 't': 'ietfk',
 'j': 'lgj',
 'd': 'bpa',
 'i': 'rst',
 '-': '-',
 'n': 'svw',
 '4': '275',
 '0': '0',
 '1': '1',
 'b': 'hd',
 'ü': 'ü',
 'z': 'z',
 'p': 'b',
 '.': '.',
 'ä': 'ä',
 'a': 'ö',
 'ß': 'ß',
 'x': 'd',
 'u': 'w',
 ',': ',',
 'g': 'g',
 'l': 'l',
 'k': 'k',
 'ö': 'u',
 'c': 'abcdefghijklmnopqrstuvwxyz',
 'e': 'abcdefghijklmnopqrstuvwxyz',
 'f': 'abcdefghijklmnopqrstuvwxyz',
 'h': 'abcdefghijklmnopqrstuvwxyz',
 'm': 'abcdefghijklmnopqrstuvwxyz',
 'q': 'abcdefghijklmnopqrstuvwxyz',
 'r': 'abcdefghijklmnopqrstuvwxyz',
 's': 'abcdefghijklmnopqrstuvwxyz',
 'v': 'abcdefghijklmnopqrstuvwxyz',
 'w': 'abcdefghijklmnopqrstuvwxyz',
 'y': 'abcdefghijklmnopqrstuvwxyz'}

In [23]:
def is_valid_german_word(word):
    """
    Validates if a word follows German letter combination rules.
    Returns True if the word is valid, False if it contains impossible combinations.
    """
    word = word.lower()
    
    # 1. "dt" at the beginning of a word
    if word.startswith('dt'):
        return False
    
    # 2. "ck" at the beginning of a word
    if word.startswith('ck'):
        return False
    
    # 3. "tz" at the end without a vowel before it
    if word.endswith('tz') and len(word) > 2:
        if word[-3] not in 'aeiouäöü':
            return False
    
    # 4. "pf", "sch", "st", "sp" at the end without a vowel before them
    end_groups = ['pf', 'st', 'sp']
    if any(word.endswith(group) and len(word) > len(group) and word[-(len(group)+1)] not in 'aeiouäöü' for group in end_groups):
        return False
    if word.endswith('sch') and len(word) > 3 and word[-4] not in 'aeiouäöü':
        return False
    
    # 5. "ä", "ö", "ü" followed immediately by another vowel
    for i in range(len(word) - 1):
        if word[i] in 'äöü' and word[i+1] in 'aeiouäöüy':
            return False
    
    # 6. "ng" at the beginning of a word
    if word.startswith('ng'):
        return False
    
    # 7. "q" without a following "u"
    for i in range(len(word) - 1):
        if word[i] == 'q' and word[i+1] != 'u':
            return False
    if word.endswith('q'):
        return False
    
    # 8 & 9. "dh" and "th" in native German words
    # This is hard to check programmatically as it depends on word origin
    
    # 10. "aa", "ee", "oo" are rare in native German words
    # Since these are rare but not impossible, we'll skip this check
    
    # 11. "y" is rare in native German words
    # Since it's rare but not impossible, we'll skip this check
    
    # 12. Triple consonants without a hyphen
    consonants = 'bcdfghjklmnpqrstvwxz'
    for i in range(len(word) - 2):
        if (word[i] in consonants and 
            word[i+1] in consonants and 
            word[i+2] in consonants and
            word[i] == word[i+1] == word[i+2]):
            return False
    
    return True

def produce_variants(word, rules=rules):
    result = []
    vectors = [list(rules[c]) for c in word]
    for prod in it.product(*vectors):
        word = ''.join(prod)
        if is_valid_german_word(word):
            yield word


([var for var in produce_variants('dot')])

['bei',
 'bee',
 'bet',
 'bef',
 'bek',
 'bni',
 'bne',
 'bnt',
 'bnf',
 'bnk',
 'bmi',
 'bme',
 'bmt',
 'bmf',
 'bmk',
 'bai',
 'bae',
 'bat',
 'baf',
 'bak',
 'bui',
 'bue',
 'but',
 'buf',
 'buk',
 'bpi',
 'bpe',
 'bpt',
 'bpk',
 'boi',
 'boe',
 'bot',
 'bof',
 'bok',
 'bci',
 'bce',
 'bct',
 'bcf',
 'bck',
 'pei',
 'pee',
 'pet',
 'pef',
 'pek',
 'pni',
 'pne',
 'pnt',
 'pnf',
 'pnk',
 'pmi',
 'pme',
 'pmt',
 'pmf',
 'pmk',
 'pai',
 'pae',
 'pat',
 'paf',
 'pak',
 'pui',
 'pue',
 'put',
 'puf',
 'puk',
 'ppi',
 'ppe',
 'ppt',
 'ppk',
 'poi',
 'poe',
 'pot',
 'pof',
 'pok',
 'pci',
 'pce',
 'pct',
 'pcf',
 'pck',
 'aei',
 'aee',
 'aet',
 'aef',
 'aek',
 'ani',
 'ane',
 'ant',
 'anf',
 'ank',
 'ami',
 'ame',
 'amt',
 'amf',
 'amk',
 'aai',
 'aae',
 'aat',
 'aaf',
 'aak',
 'aui',
 'aue',
 'aut',
 'auf',
 'auk',
 'api',
 'ape',
 'apt',
 'apf',
 'apk',
 'aoi',
 'aoe',
 'aot',
 'aof',
 'aok',
 'aci',
 'ace',
 'act',
 'acf',
 'ack']

In [7]:
model_name = "dbmdz/german-gpt2"
model = transformers.GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [ ]:
def decipher(text, model=model, tokenizer=tokenizer, rules=rules):
    result = '. '
    text_list = text.split(' ')
    text_list = [word.strip('.').strip('\n').strip(',') for word in text_list]
    
    for word in text_list:
        if not word:
            continue
        inputs = tokenizer(result, return_tensors='pt')
        
        # Get model outputs (logits)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        
        # Convert logits to probabilities using softmax
        probabilities = F.softmax(logits, dim=-1)

        variants = produce_variants(word.lower(), rules=rules)
        max_prob = 0.0
        best_var = ' '
        for variant in variants:
            tmp = ' '
            if word.istitle():
                tmp += variant.capitalize()
            else:
                tmp += variant
            #next_token_id = tokenizer.encode(tmp)[0]  # Token ID for the next token "example"
            #probability = probabilities[0, -1, next_token_id].item()
            next_token_ids = tokenizer.encode(tmp)
            probability = 1.0
            for token_id in next_token_ids:
                # Assuming we are calculating the probability of the next tokens after the input text
                probability *= probabilities[0, -1, token_id].item()

            if probability > max_prob:
                max_prob = probability
                best_var = tmp
        result += best_var
        print(result)
    return result[3:]

decipher('Xoob noi ooiitoit, nooo bto Küoitjtobo Iotojjtjooz ott tbioo Uoiboiiojoo tüi boi ntitoojjo')

.  Doch
.  Doch was
.  Doch was passiert
.  Doch was passiert wenn
.  Doch was passiert wenn die
.  Doch was passiert wenn die Künstliche
